In [ ]:
# loading libraries
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist

from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from sklearn.preprocessing import LabelBinarizer
from keras.optimizers import *
from keras import backend as K
from keras.regularizers import l2
from keras.utils.vis_utils import plot_model


In [ ]:
!pip install gdal

In [ ]:
import sys

sys.path

#sys.path.append('c:\programdata\anaconda3\envs\tf2.8\lib\site-packages (3.4.1)')

In [ ]:
from osgeo import gdal
import numpy as np


def read_as_array(path,file_name):

    gdal_file=gdal.Open(path+'/'+file_name)

    array=np.zeros((gdal_file.RasterYSize,gdal_file.RasterXSize,gdal_file.RasterCount))
    
    for i in range(gdal_file.RasterCount):

        gdal_file_band=gdal_file.GetRasterBand(i+1)

        array[:,:,i]=gdal_file_band.ReadAsArray()
        
    return(array)     

path='D:/Nafiseh/flood_proposal'

#preflood_image_name,postflood_image_name='0417_clip_2.tif','20190428_coregis_stack_bands.tif'

#preflood_image_name,postflood_image_name='subset_1_of_S1A_IW_GRDH_1SDV_20190408T225218_20190408T225243_Cal_Spk_TC_2.tif','subset_0_of_S1A_IW_GRDH_1SDV_20190425T230019_20190425T230055_Cal_Spk_TC_4_2.tif'
#coh_image_name='S1A_IW_SLC__1SDV_20190408T225217_20190502_20190514_20190526_split_Orb_Stack_esd_coh_deb_flt_ML_TC_Stack.tif'


#preflood_array=read_as_array(path,preflood_image_name)

#postflood_array=read_as_array(path,postflood_image_name)

#postflood_array=np.delete(postflood_array,-1,axis=1)

#stack_image_name='corgistration_intensity_coh_Stack.tif'

#stack_image_name='subset_0_of_S1A_IW_GRDH_1SDV_Cal_Spk_TC_Stack_subset.tif'
#stack_image_name_2='subset_1_of_S1A_IW_GRDH_1SDV_20190408T225218_20190507T23009_Cal_Spk_TC_Stack_2_subset.tif'

#train_name='collocate_int_coh_leverkhuzen_2.tif'

train_name='stack_train_RCM_3_20211118_1130.tif'

#test_name='collocate_S1B_int_coh.tif'
test_name='stack_test_RCM_2_3_2021119_1130.tif'


#GRT_Mask_name='flood_mask_mosaic_to_new_raster.tif'

#MNDWI_Mask_name='S2_MNDWI_flood_mask_clip3_binary_region_group_nibble_mask_nibble_boundary_clean.tif'

#stack_array=read_as_array(path,stack_image_name)
#stack_array_2=read_as_array(path,stack_image_name_2)
#label=read_as_array(path,MNDWI_Mask_name)
#label_test=read_as_array(path,GRT_Mask_name)
#label[label>=1]=1

train_array_int=read_as_array(path,train_name)
##temp=train_array_int
#temp[temp==0]=10**-10
#stack_train_int=np.log10(temp) #vv channel for S1 #HH and HV channels for RCM
stack_train_int=train_array_int

test_array_int=read_as_array(path,test_name)
temp_=test_array_int
##temp_=np.copy(test_array)
#temp_[temp_==0]=10**-10
#stack_test_int=np.log10(temp_) #vv channel for S1 #HH and HV channels for RCM
stack_test_int=test_array_int

#GRD_SLC_stack_train_s=stack_train_int[:,:,[0,1,2,3,5,6,8,9]]
GRD_SLC_stack_train_s=stack_train_int

#GRD_SLC_stack_test_s=stack_test_int[:,:,[0,1,2,3,5,6,8,9]]
GRD_SLC_stack_test_s=stack_test_int

#preflood_array_=np.concatenate((np.delete(np.repeat(stack_array[:2000,500:2000,2:4],2,axis=2),-1,axis=2),coh_co_vv_array_),axis=2) #VH;VV;VH db
#postflood_array_=np.concatenate((np.delete(np.repeat(stack_array[:2000,500:2000,14:16],2,axis=2),-1,axis=2),coh_post_vv_array_),axis=2)
#preflood_array_=np.concatenate((stack_array[:,:,2:4],coh_co_vv_array_),axis=2)
#postflood_array_=np.concatenate((stack_array[:,:,-3:-1],coh_post_vv_array_),axis=2)

#intensity_array_vv=stack_array[500:2000,500:2000,[7,3]]

#coh_array_vv=np.expand_dims(stack_array[:,:,[8,10]],axis=2)  #coherency_co and coherency_post; vv; post flood coherency was considered as reference

#coherency_array_vv=np.expand_dims((intensity_array_vv,coh_vv_array_))
#postflood_array_=np.expand_dims(stack_array_2[500:2000,500:2000,7],axis=2)
#label=stack_array[780:1080,0:300,-1] #flood mask for 25th April



#label_test=np.round(stack_array_2[500:2000,500:2000,-1])
#label_test=np.round(stack_array_2[780:1080,0:300,-1])
#label_test[np.isnan(label_test)]=0
#label_test[np.where(label_test==0)[0]]=0
#label_test[np.where(label_test>1)[0]]=1 #making label binary (the original labels were not binary maybe because of the co-registration); flood mask for 5th May

#print(GRD_SLC_stack_train_s.shape,np.unique(label),np.unique(label_test))
print(GRD_SLC_stack_train_s.shape,GRD_SLC_stack_test_s.shape)

# Comparison with Simple Log-Ratio method

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix

from sklearn.metrics import classification_report
from scipy.ndimage import zoom
import skimage


fig,ax=plt.subplots(2,1,figsize=(10,5))

axes=np.ravel(ax)

gt=read_as_array('D:/Nafiseh/flood_proposal/','hydrography_area_mask_leverkhuzen_clip.tif')

log_ratio=np.log10(test_array_int[:,:,3]/test_array_int[:,:,1]) #log10(co/pre)

log_ratio[~np.isfinite(log_ratio)]=0

otsu_threshold=skimage.filters.threshold_otsu(log_ratio)

log_ratio[log_ratio>otsu_threshold]=1

log_ratio[log_ratio<=otsu_threshold]=0

log_ratio_r=zoom(log_ratio,[gt.shape[0]/log_ratio.shape[0],gt.shape[1]/log_ratio.shape[1]],order=0)

im=axes[0].imshow(log_ratio_r)

axes[0].axis('off')

im=axes[1].imshow(gt)

axes[1].axis('off')

cax_colorbar=fig.add_axes([0.65, 0.15, 0.05, 0.7])

cb=fig.colorbar(im,ticks=[0,1],cax=cax_colorbar)

cb.set_ticklabels(['background','flood/water bodies'],fontsize='x-large')

plt.show()

In [ ]:
conf_log_ratio=confusion_matrix(np.ravel(gt),np.ravel(log_ratio_r))
print(classification_report(np.ravel(gt),np.ravel(log_ratio_r)))

In [ ]:
!pip install scikit-image

In [ ]:
%matplotlib inline
import skimage

from scipy.ndimage import zoom

import matplotlib.pyplot as plt

def getThreshold(data):
    
    n = data.shape[2]
    
    threshold = []
    
    for i in range(n):
        
        band_data = data[:,:,i]

        # calculate threshold using Otsu method
        threshold_otsu = skimage.filters.threshold_otsu(band_data)
        # calculate threshold using minimum method
        threshold_minimum = skimage.filters.threshold_minimum(band_data)
        # get number of pixels for both thresholds
        numPixOtsu = len(band_data[abs(band_data - threshold_otsu) < 0.1])
        numPixMinimum = len(band_data[abs(band_data - threshold_minimum) < 0.1])

        # if number of pixels at minimum threshold is less than 1% of number of pixels at Otsu threshold
        if abs(numPixMinimum/(numPixOtsu+1)) < 0.01:
            # adjust band data according
            if threshold_otsu < threshold_minimum:
                
                band_data = band_data[band_data < threshold_minimum]
                
                if np.any(band_data!=np.mean(np.ravel(band_data))):
                    
                    threshold_minimum = skimage.filters.threshold_minimum(band_data)
            else:
                
                band_data = band_data[band_data > threshold_minimum]
                threshold_minimum = skimage.filters.threshold_minimum(band_data)

            numPixMinimum = len(band_data[abs(band_data - threshold_minimum) < 0.1])

        # select final threshold
        if abs(numPixMinimum/(numPixOtsu+1)) < 0.01:
            threshold.append(threshold_otsu)
        else:
            threshold.append(threshold_minimum)

    return threshold

dem_name='Abbotsford_dem_2012.tif'
#dem_name='dem_gatineau.tif'
#dem_name='dem_leverkhuzen_SRTMGL1.tif'

dem=read_as_array(path,dem_name)

dem_row,dem_col=dem.shape[0],dem.shape[1]

#stack_train_int_coh=GRD_SLC_stack_train_s[:,:,[3,7]]
stack_train_int_coh=GRD_SLC_stack_train_s #HH and HV bands

#stack_test_int_coh=GRD_SLC_stack_test_s[:,:,[3,7]]
stack_test_int_coh=GRD_SLC_stack_test_s #HH and HV bands


row=stack_train_int_coh.shape[0]
col=stack_train_int_coh.shape[1]

row1=stack_test_int_coh.shape[0]
col1=stack_test_int_coh.shape[1]


dem_r=zoom(dem,(row/dem_row,col/dem_col,1),order=0)

dem_r_test=zoom(dem,(row1/dem_row,col1/dem_col,1),order=0)

dem_threshold=getThreshold(dem_r)

int_thresholds=getThreshold(np.divide(stack_train_int_coh,256))

int_thresholds_test=getThreshold(np.divide(stack_test_int_coh,256))

#mask_vv_pre=np.zeros((row,col))
mask_vv_co=np.zeros((row,col))
mask_vv_coh_co=np.zeros((row,col))
mask_intersection=np.zeros((row,col))
mask_union=np.zeros((row,col))
mask_union_test=np.zeros((row1,col1))
mask_intersection_test=np.zeros((row1,col1))
mask_vv_co_test=np.zeros((row1,col1))
mask_vv_coh_co_test=np.zeros((row1,col1))

##mask_vv_pre[stack_train_int[:,:,0]<int_thresholds[0]]=1

mask_vv_co[stack_train_int_coh[:,:,0]<int_thresholds[0]]=1 #co_event vv:sentinel-1; hh:RCM

mask_vv_coh_co[stack_train_int_coh[:,:,1]<int_thresholds[1]]=1 #co_event coh:sentinel-1; hv (intensity):RCM

mask_intersection[(mask_vv_co==1) & (mask_vv_coh_co==1) & (np.squeeze(dem_r,axis=2)<dem_threshold[0])]=1

mask_union[((mask_vv_co==1) | (mask_vv_coh_co==1)) & (np.squeeze(dem_r,axis=2)<dem_threshold[0])]=1

mask_vv_co_test[stack_test_int_coh[:,:,0]<int_thresholds_test[0]]=1 #co_event

mask_vv_coh_co_test[stack_test_int_coh[:,:,1]<int_thresholds_test[1]]=1

mask_union_test[((mask_vv_co_test==1) | (mask_vv_coh_co_test==1)) & (np.squeeze(dem_r_test,axis=2)<dem_threshold[0])]=1

mask_intersection_test[(mask_vv_co_test==1) & (mask_vv_coh_co_test==1) & (np.squeeze(dem_r_test,axis=2)<dem_threshold[0])]=1

fig,ax=plt.subplots(2,3,figsize=(10,5))

ax[0,0].imshow(mask_vv_co)

ax[0,0].axis('off')

#ax[0,0].set_title('VV mask co event (2021/07/16)')
ax[0,0].set_title('HH mask co event (2021/11/18)')

ax[0,1].imshow(mask_vv_coh_co)

ax[0,1].axis('off')

#ax[0,1].set_title('Co event coherency VV mask \n (2021/06/24-2021/07/06)')

ax[0,1].set_title('HV mask co event \n (2021/11/18)')

ax[0,2].imshow(mask_union)

ax[0,2].axis('off')

ax[0,2].set_title('Union Mask')

ax[1,0].imshow(mask_vv_co_test)

ax[1,0].axis('off')

#ax[1,0].set_title('VV mask co event (2021/07/18)')

ax[1,0].set_title('HH mask co event (2021/11/19)')

ax[1,1].imshow(mask_vv_coh_co_test)

#ax[1,1].set_title('Co event coherency VV mask \n (2021/07/07-2021/07/19)')

ax[1,1].set_title('HV mask co event \n (2021/11/19)')

ax[1,1].axis('off')

ax[1,2].imshow(mask_union_test)

ax[1,2].set_title('Union Mask')

ax[1,2].axis('off')

plt.tight_layout()

plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig,ax=plt.subplots(1,3,figsize=(10,5))

ax[0].imshow(mask_intersection,cmap='magma') #25th April flood mask

ax[0].axis('off')

ax[1].imshow(mask_intersection_test,cmap='magma') #7th May flood mask

ax[1].axis('off')

ax[2].imshow(mask_intersection_test,cmap='magma')

ax[2].axis('off')

plt.show()

# KMeans Clustering

In [ ]:
from sklearn.cluster import KMeans

def vectorize_3d_array(three_d_array):
    
    array=np.zeros((three_d_array.shape[0]*three_d_array.shape[1],three_d_array.shape[2]))
    
    for i in range(three_d_array.shape[2]):
    
        array[:,i]=np.ravel(three_d_array[:,:,i])
        
    return(array)     


preflood_1d_array=vectorize_3d_array(preflood_array_)

#postflood_1d_array=vectorize_3d_array(postflood_array_)

NDWI_pre=np.nan_to_num(np.log(np.divide((preflood_1d_array[:,0]-preflood_1d_array[:,1]),(preflood_1d_array[:,0]+preflood_1d_array[:,1]))))

#NDWI_post=np.nan_to_num(np.divide((postflood_1d_array[:,1]-postflood_1d_array[:,3]),(postflood_1d_array[:,1]+postflood_1d_array[:,3])))

pre_kmeans=KMeans(n_clusters=3,random_state=0).fit(NDWI_pre.reshape(-1,1))

preflood_cluster=pre_kmeans.predict(NDWI_pre.reshape(-1,1))

#post_kmeans=KMeans(n_clusters=3,random_state=0).fit(NDWI_post.reshape(-1,1))

#postflood_cluster=post_kmeans.predict(NDWI_post.reshape(-1,1))

# Loading Clustering Result

In [ ]:
import numpy as np
path='D:/flood_proposal'
#np.save(path+'/postflood_cluster.npy',postflood_cluster)
#np.save(path+'/preflood_cluster.npy',preflood_cluster)
#postflood_cluster=np.load(path+'/postflood_cluster.npy')
preflood_cluster=np.load(path+'/preflood_cluster.npy')

# Visualizing Clustering Result

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


fig=plt.figure(figsize=(5,5))

plt.imshow(np.reshape(postflood_array_,(postflood_array_.shape[0],postflood_array_.shape[1])),cmap='magma')

plt.axis('off')

plt.show()

In [ ]:
kmeans_preflood_clustering=np.reshape(preflood_cluster,(preflood_array_.shape[0],preflood_array_.shape[1]))
kmeans_postflood_clustering=np.reshape(postflood_cluster,(preflood_array_.shape[0],preflood_array_.shape[1]))

# Train Data Preparation (Part 1)

In [ ]:
from scipy.ndimage import zoom

def prepare_train_data(flood_array,label):

    row,col=flood_array.shape[0],flood_array.shape[1]
    
    neigh_size=256

    half_size=np.int((neigh_size-1)/2)

    train_sample=[]

    label_new=[]

    for i in range(half_size,row-half_size+1,neigh_size):

        for j in range(half_size,col-half_size+1,neigh_size):

            train_sample.append(flood_array[i-half_size:i+half_size+1,j-half_size:j+half_size+1,:])

            label_new.append(label[i-half_size:i+half_size+1,j-half_size:j+half_size+1])

            #central_pixel.append((i,j))

    return(train_sample,label_new)

def prepare_test_data(postflood_array,label_test):
    
    neigh_size=256
    
    half_size=np.int((neigh_size-1)/2)
    
    test_sample=[]
    
    label_new=[]
    
    #postflood_array_pad=np.pad(postflood_array,((half_size,half_size),(half_size,half_size),(0,0)),'symmetric')
    
    #label_test_pad=np.pad(label_test,((half_size,half_size),(half_size,half_size),(0,0)),'symmetric')
    
    row,col=postflood_array.shape[0],postflood_array.shape[1]
    
    for i in range(half_size,row-half_size+1,neigh_size):
        
        for j in range(half_size,col-half_size+1,neigh_size):
            
            test_sample.append(postflood_array[i-half_size:i+half_size+1,j-half_size:j+half_size+1,:])
            
            label_new.append(label_test[i-half_size:i+half_size+1,j-half_size:j+half_size+1])

            
    return(test_sample,label_new)       

#train_sample,label=prepare_train_data(intensity_array_vv,label)            
#test_sample,label_test=prepare_test_data(postflood_array_,label_test)


##label=zoom(label,(300/label.shape[0],300/label.shape[1],1),order=0)
#mask_union=np.expand_dims(mask_union,axis=2)
train_sample,label=prepare_train_data(GRD_SLC_stack_train_s,mask_union)
#mask_union_test=np.expand_dims(mask_union_test,axis=2)
##label_test=zoom(label_test,(300/label_test.shape[0],300/label_test.shape[1],1),order=0)
test_sample,label_test=prepare_test_data(GRD_SLC_stack_test_s,mask_union_test)

# Train Data Preparation (Part 2)

In [ ]:
from scipy.ndimage import zoom

import random

def resample_data(data_sample,label,mode):

    data_resampled=[]
    
    label_new=[]
    
    patch_size=256
    
    #if label.ndim==2:
       
       #label=np.ravel(label) 

    for k in range(len(data_sample)):
        
        #print(data_sample[k].shape)
        
        if data_sample[k].shape<(patch_size,patch_size):

            data_resampled.append(zoom(data_sample[k],zoom=[patch_size/data_sample[k].shape[0],patch_size/data_sample[k].shape[1],1],order=3))

            label_new.append(zoom(label[k],zoom=[patch_size/label[k].shape[0],patch_size/label[k].shape[1]],order=1))
            
            #continue
        
        elif data_sample[k].shape==(patch_size,patch_size):
            
            data_resampled.append(data_sample[k])
            
            label_new.append(label[k])
            
        elif data_sample[k].shape>(patch_size,patch_size):
            
            for i in range(round(data_sample[k].shape[0]/patch_size)):
                
                for j in range(round(data_sample[k].shape[1]/patch_size)):

                
                    temp=data_sample[k][i*patch_size:(i+1)*patch_size,j*patch_size:(j+1)*patch_size]
                
                                
                    if temp.shape<(patch_size,patch_size):

                    
                        data_resampled.append(zoom(temp,zoom=[patch_size/temp.shape[0],patch_size/temp.shape[1],1],order=3))
                        #continue
                        label_new.append(zoom(label[k],zoom=[patch_size/label[k].shape[0],patch_size/label[k].shape[1]],order=1)) 

                    else:
                    
                        data_resampled.append(temp) 
                    
                        label_new.append(zoom(label[k],zoom=[patch_size/label[k].shape[0],patch_size/label[k].shape[1]],order=1))    
                
                   

    data_resampled=np.stack(data_resampled) 

    label_new=np.stack(label_new)

    print(f' {mode} Data Shape: {data_resampled.shape}, {mode} Label Shape: {label_new.shape}')
    
    return (data_resampled,label_new)

train_resampled,label_new=resample_data(train_sample,label,'Train')   

test_resampled,label_test_new=resample_data(test_sample,label_test,'Test')

# Data Augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator


def data_augmentation(data,label_new,image_count=150):

        augmented_images=[]
        
        label_aug=[]

        datagen_hflip=ImageDataGenerator(rotation_range=0.1, horizontal_flip=True)
        
        for sn in range(data.shape[0]):

            it=datagen_hflip.flow(np.expand_dims(data[sn,:,:,:],axis=0),batch_size=1)

            for j in range(image_count):
    
                augmented_images.append(np.squeeze(it.next(),axis=0))
        
                label_aug.append(label_new[sn,:,:])
        
        return(np.stack(augmented_images),np.stack(label_aug)) 
    
#positive_idx,negative_idx=np.where(np.array(label_new)==0),np.where(np.array(label_new)==1)

#train_resampled_aug_pos,label_aug_pos=data_augmentation(train_resampled,label_new,positive_idx,20) #positive is background

train_resampled_aug,label_aug=data_augmentation(train_resampled,label_new)

train_resampled_aug=np.concatenate((train_resampled,train_resampled_aug),axis=0)

label_aug=np.concatenate((label_new,label_aug),axis=0)

In [ ]:
def normalize_data(data_resampled,mode):
    
    for i in range(data_resampled.shape[0]):

            if len(data_resampled.shape)==4:
                
                for j in range(data_resampled.shape[3]):

                    a=data_resampled[i,:,:,j]

                    min_,max_=np.min(np.ravel(data_resampled[i,:,:,j])),np.max(np.ravel(data_resampled[i,:,:,j]))

                    data_resampled[i,:,:,j]=(a-min_)/(max_-min_)
                
            elif len(data_resampled.shape)==2: 
                
                a=data_resampled[i,:]

                min_,max_=np.min(np.ravel(data_resampled[i,:])),np.max(np.ravel(data_resampled[i,:]))

                data_resampled[i,:]=(a-min_)/(max_-min_)

    print(f' {mode} Data Shape: {data_resampled.shape}') 
    
    return(data_resampled)

train_resampled_aug=normalize_data(train_resampled_aug,'Train')

test_resampled=normalize_data(test_resampled, 'Test')

# Splitting data into train and validation

In [ ]:
import random

from sklearn.utils import resample

from sklearn.model_selection import train_test_split

train_resampled,label_new=train_resampled_aug,label_aug

#positive here is the background

train_resampled_,val_resampled,train_label,val_label=train_test_split(train_resampled,label_new,test_size=0.2,random_state=1)

print(f' Train Data Shape: {train_resampled_.shape}, Validation Data Shape: {val_resampled.shape}')

print(f' \n Train Label Shape: {train_label.shape}, Validation Label Shape: {val_label.shape}')

In [ ]:
import matplotlib.pyplot as plt

fig,axes=plt.subplots(2,2,figsize=(10,5))

for i,ax in zip(range(4),axes.ravel()):
    
    ax.imshow(train_resampled[i,:,:,1])
    
    ax.axis('off')

plt.show()    

In [ ]:
# [ Siamese Segmentation models ]
#
# Altered code from:
# https://github.com/qubvel/segmentation_models
# more specifically combined from files:
# - https://github.com/qubvel/segmentation_models/blob/master/segmentation_models/unet/builder.py
# - https://github.com/qubvel/segmentation_models/blob/master/segmentation_models/unet/blocks.py
# under commit https://github.com/qubvel/segmentation_models/commit/9c68d81d66e4fb856770a87b450a43bb2ae6ddba

from keras.layers import Conv2D
from keras.layers import Activation
from keras.models import Model

from segmentation_models.utils import freeze_model
from segmentation_models.utils import legacy_support
from segmentation_models.backbones import get_backbone, get_feature_layers

from segmentation_models.unet.blocks import Transpose2D_block
from segmentation_models.utils import get_layer_number, to_tuple

from keras.layers import Concatenate
from segmentation_models.unet.blocks import UpSampling2D, handle_block_names, ConvRelu

import keras
from keras.layers import Input
from keras.models import load_model

old_args_map = {
    'freeze_encoder': 'encoder_freeze',
    'skip_connections': 'encoder_features',
    'upsample_rates': None,  # removed
    'input_tensor': None,  # removed
}


#@legacy_support(old_args_map)
def SiameseUnet(backbone_name='vgg16',
         input_shape=(None, None, 3),
         classes=1,
         activation='sigmoid',
         encoder_weights='imagenet',
         encoder_freeze=False,
         encoder_features='default',
         decoder_block_type='upsampling',
         decoder_filters=(256, 128, 64, 32, 16),
         decoder_use_batchnorm=True,
         **kwargs):
    """ Unet_ is a fully convolution neural network for image semantic segmentation
        Args:
            backbone_name: name of classification model (without last dense layers) used as feature
                extractor to build segmentation model.
            input_shape: shape of input data/image ``(H, W, C)``, in general
                case you do not need to set ``H`` and ``W`` shapes, just pass ``(None, None, C)`` to make your model be
                able to process images af any size, but ``H`` and ``W`` of input images should be divisible by factor ``32``.
            classes: a number of classes for output (output shape - ``(h, w, classes)``).
            activation: name of one of ``keras.activations`` for last model layer
                (e.g. ``sigmoid``, ``softmax``, ``linear``).
            encoder_weights: one of ``None`` (random initialization), ``imagenet`` (pre-training on ImageNet).
            encoder_freeze: if ``True`` set all layers of encoder (backbone model) as non-trainable.
            encoder_features: a list of layer numbers or names starting from top of the model.
                Each of these layers will be concatenated with corresponding decoder block. If ``default`` is used
                layer names are taken from ``DEFAULT_SKIP_CONNECTIONS``.
            decoder_block_type: one of blocks with following layers structure:
                - `upsampling`:  ``Upsampling2D`` -> ``Conv2D`` -> ``Conv2D``
                - `transpose`:   ``Transpose2D`` -> ``Conv2D``
            decoder_filters: list of numbers of ``Conv2D`` layer filters in decoder blocks
            decoder_use_batchnorm: if ``True``, ``BatchNormalisation`` layer between ``Conv2D`` and ``Activation`` layers
                is used.
        Returns:
            ``keras.models.Model``: **Unet**
        .. _Unet:
            https://arxiv.org/pdf/1505.04597
    """

    load_weights_from = None
    if encoder_weights is not "imagenet" and encoder_weights is not None:
        load_weights_from = encoder_weights
        encoder_weights = None


    backbone = get_backbone(backbone_name,
                            input_shape=input_shape,
                            input_tensor=None,
                            weights=encoder_weights,
                            include_top=False)

    if load_weights_from is not None:
        model_to_load_weights_from = load_model(load_weights_from)

        # now let's assume that this loaded model had its own "top" upsampling section trained on another task
        # let's transplant what we can, that is the backbone encoder

        output = model_to_load_weights_from.layers[len(backbone.layers)-1].output  # remove activation and last conv layer
        transplant = keras.models.Model(model_to_load_weights_from.input, output)
        #transplant.summary()

        transplant.save("transplant.h5") # hacky way
        backbone.load_weights("transplant.h5")

        # Check if the weights have been loaded
        """
        inspect_i = 0
        import numpy as np
        w1 = np.asarray(transplant.get_weights()[inspect_i])
        print(w1)
        w2 = np.asarray(backbone.get_weights()[inspect_i])
        print(w2)
        """
        print("Loaded weights into ",backbone_name,"from",load_weights_from)

    if encoder_features == 'default':
        encoder_features = get_feature_layers(backbone_name, n=4)

    model = build_siamese_unet(backbone,
                       classes,
                       encoder_features,
                       decoder_filters=decoder_filters,
                       block_type=decoder_block_type,
                       activation=activation,
                       n_upsample_blocks=len(decoder_filters),
                       upsample_rates=(2, 2, 2, 2, 2),
                       use_batchnorm=decoder_use_batchnorm,
                       input_shape=input_shape)

    # lock encoder weights for fine-tuning
    if encoder_freeze:
        freeze_model(backbone)

    model.name = 'u-{}'.format(backbone_name)

    return model



def Siamese_Upsample2D_block(filters, stage, kernel_size=(3,3), upsample_rate=(2,2),
                     use_batchnorm=False, skip_a=None, skip_b=None):

    def layer(input_tensor):

        conv_name, bn_name, relu_name, up_name = handle_block_names(stage)

        x = UpSampling2D(size=upsample_rate, name=up_name)(input_tensor)

        if skip_a is not None and skip_b is not None:
            x = Concatenate()([x, skip_a, skip_b]) # siamese concatenation

        x = ConvRelu(filters, kernel_size, use_batchnorm=use_batchnorm,
                     conv_name=conv_name + '1', bn_name=bn_name + '1', relu_name=relu_name + '1')(x)

        x = ConvRelu(filters, kernel_size, use_batchnorm=use_batchnorm,
                     conv_name=conv_name + '2', bn_name=bn_name + '2', relu_name=relu_name + '2')(x)

        return x
    return layer


def build_siamese_unet(backbone, classes, skip_connection_layers,
               decoder_filters=(256,128,64,32,16),
               upsample_rates=(2,2,2,2,2),
               n_upsample_blocks=5,
               block_type='upsampling',
               activation='sigmoid',
               use_batchnorm=True,
               input_shape=(None, None, 3)):

    verbose = False
    if verbose:
        print("Entered build_unet with arguments:")
        print("backbone",backbone)
        #print("---\n")
        #backbone.summary()
        #print("---\n")


        print("classes",classes)
        print("skip_connection_layers",skip_connection_layers)
        print("decoder_filters",decoder_filters)
        print("upsample_rates",upsample_rates)
        print("n_upsample_blocks",n_upsample_blocks)
        print("block_type",block_type)
        print("activation",activation)
        print("use_batchnorm",use_batchnorm)

    input = backbone.input
    x = backbone.output

    # Prepare for multiple heads in siamese nn:

    skip_connection_idx = ([get_layer_number(backbone, l) if isinstance(l, str) else l
                               for l in skip_connection_layers])
    if verbose:
        print("skip_connection_idx", skip_connection_idx)

    skip_connections = []
    for idx in skip_connection_idx:
        skip_connection = backbone.layers[idx].output
        skip_connections.append(skip_connection)

    if verbose:
        print("skip_connections layers", len(skip_connections), skip_connections)
    #4 layers
    # 'stage4_unit1_relu1/Relu:0' shape=(?, 16, 16, 256)
    # 'stage3_unit1_relu1/Relu:0' shape=(?, 32, 32, 128)
    # 'stage2_unit1_relu1/Relu:0' shape=(?, 64, 64, 64)
    # 'relu0/Relu:0'              shape=(?, 128, 128, 64)

    siamese_backbone_model_encode = Model(inputs=[input], outputs=[x]+skip_connections)

    if verbose:
        print("siamese_model_encode.input", siamese_backbone_model_encode.input)
        print("siamese_model_encode.output", siamese_backbone_model_encode.output) # x and the (now 4) skip connections

    # Then merging
    input_a = Input(shape=(input_shape[0], input_shape[1], input_shape[2]))
    input_b = Input(shape=(input_shape[0], input_shape[1], input_shape[2]))

    branch_a_outputs = siamese_backbone_model_encode([input_a])
    branch_b_outputs = siamese_backbone_model_encode([input_b])

    branch_a = branch_a_outputs[0]
    branch_b = branch_b_outputs[0]

    x = Concatenate(name="concatHighLvlFeat")([branch_a, branch_b]) # both inputs, in theory 8x8x512 + 8x8x512 -> 8x8x1024

    skip_connection_outputs_a = branch_a_outputs[1:]
    skip_connection_outputs_b = branch_b_outputs[1:]

    if block_type == 'transpose':
        up_block = Transpose2D_block
        assert False # NOT IMPLEMENTED
    else:
        up_block = Siamese_Upsample2D_block

    for i in range(n_upsample_blocks):
        skip_connection_a = None
        skip_connection_b = None
        if i < len(skip_connection_idx): # also till len(skip_connection_outputs_a)
            skip_connection_a = skip_connection_outputs_a[i]
            skip_connection_b = skip_connection_outputs_b[i]

        upsample_rate = to_tuple(upsample_rates[i])

        x = up_block(decoder_filters[i], i, upsample_rate=upsample_rate,
                     skip_a=skip_connection_a, skip_b=skip_connection_b, use_batchnorm=use_batchnorm)(x)

    x = Conv2D(classes, (3,3), padding='same', name='final_conv')(x)
    x = Activation(activation, name=activation)(x)

    #model = Model(input, x)
    full_model = Model(inputs=[input_a, input_b], outputs=x)

    return full_model



# There is support for all of these (with weights from ImageNet included) ... qubvel/segmentation_models is awesome!
# VGG           'vgg16' 'vgg19'
# ResNet	    'resnet18' 'resnet34' 'resnet50' 'resnet101' 'resnet152'
# SE-ResNet	    'seresnet18' 'seresnet34' 'seresnet50' 'seresnet101' 'seresnet152'
# ResNeXt	    'resnext50' 'resnet101'
# SE-ResNeXt	'seresnext50' 'seresnet101'
# SENet154	    'senet154'
# DenseNet	    'densenet121' 'densenet169' 'densenet201'
# Inception	    'inceptionv3' 'inceptionresnetv2'
# MobileNet	    'mobilenet' 'mobilenetv2'
# Performance comparison for classification: https://github.com/qubvel/classification_models

"""
BACKBONE = 'resnet34'
custom_weights_file = "model_UNet-Resnet34_DSM_in01_95percOfTrain_8batch_100ep_dsm01proper.h5" # None
custom_weights_file = "imagenet"
model = SiameseUnet(BACKBONE, encoder_weights=custom_weights_file, classes=3, activation='softmax', input_shape=(256, 256, 3))
print("Model loaded:")
print("model.input", model.input)
print("model.output", model.output)
"""
#model.summary()

# Ps: there is posibility to change the code of additional models in similar manner to get FPN, Linknet and PSPNet
# Ps2: some of these Siamese NN models end up with large amount of parameters ...
#      if we don't have much data, we should perhaps freeze some of the layers of the encoder... "encoder_freeze=False"

# Ps3: keras saves models into $ cd /home/<username>/.keras/models/

In [ ]:
SiameseUnet(backbone_name='vgg16',
         input_shape=(256, 256, 3),
         classes=1,
         activation='sigmoid',
         encoder_weights=None,
         encoder_freeze=False,
         encoder_features='default',
         decoder_block_type='upsampling',
         decoder_filters=(256, 128, 64, 32, 16),
         decoder_use_batchnorm=True)

# Embedding Generator Model

## Method 1: Resnet50

In [ ]:
from tensorflow.keras.applications import resnet

from tensorflow.keras import applications

from tensorflow.keras import layers,losses,optimizers,metrics,Model

import tensorflow as tf

tf.compat.v1.enable_eager_execution()

patch_size=32

cnn=resnet.ResNet50(weights=None,input_shape=(patch_size,patch_size)+(3,),include_top=False)

flatten=layers.Flatten()(cnn.output)

dense_1=layers.Dense(512,activation='relu')(flatten)

dense_1=layers.BatchNormalization()(dense_1)

dense_2=layers.Dense(256,activation='relu')(dense_1)

dense_2=layers.BatchNormalization()(dense_2)

output=layers.Dense(256)(dense_2)

embedding=Model(cnn.input,output,name='Embedding')

embedding.summary()

#for layer in cnn.layers:

    
    #if 'conv5_block3' in layer.name: #training just the last convolutional block
        
        #layer.trainable=True
        
    #else:
        
        #layer.trainable=False

## Method 2: VGG16

In [ ]:
from tensorflow.keras.applications import vgg16

from tensorflow.keras import layers, Model,losses, metrics, optimizers

import tensorflow as tf

tf.compat.v1.enable_eager_execution()

patch_size=32

cnn=vgg16.VGG16(weights='imagenet',input_shape=(patch_size,patch_size)+(3,),include_top=False)

flatten=layers.Flatten()(cnn.output)

dense_1=layers.Dense(512,activation='relu')(flatten)

dense_1=layers.BatchNormalization()(dense_1)

dense_2=layers.Dense(256,activation='relu')(dense_1)

dense_2=layers.BatchNormalization()(dense_2)

output=layers.Dense(256)(dense_2)

embedding=Model(cnn.input,output,name='Embedding')

#embedding.summary()

for layer in cnn.layers:

    
    if 'block5' in layer.name: #training just the last convolutional block
        
        layer.trainable=True
        
    else:
        
        layer.trainable=False

## Method 3: Densenet

In [ ]:
from tensorflow.keras.applications import densenet

from tensorflow.keras import layers, Model,losses, metrics, optimizers

import tensorflow as tf

tf.compat.v1.enable_eager_execution()

patch_size=32

cnn=densenet.DenseNet121(weights='imagenet',input_shape=(patch_size,patch_size)+(3,),include_top=False)

flatten=layers.Flatten()(cnn.output)

dense_1=layers.Dense(512,activation='relu')(flatten)

dense_1=layers.BatchNormalization()(dense_1)

dense_2=layers.Dense(256,activation='relu')(dense_1)

dense_2=layers.BatchNormalization()(dense_2)

output=layers.Dense(256)(dense_2)

embedding=Model(cnn.input,output,name='Embedding')

#embedding.summary()

for layer in cnn.layers:

    
    if 'conv5_block16' in layer.name: #training just the last convolutional block
        
        layer.trainable=True
        
    else:
        
        layer.trainable=False 

## Method : Conventional CNN Layers

In [ ]:
embedding_dim=256
patch_size=32

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Dense

embedding=Sequential([Conv2D(32,kernel_size=3,activation='relu',input_shape=(patch_size,patch_size,3)),
                     
                         Conv2D(32,kernel_size=3,activation='relu'),
                            
                         Conv2D(32,kernel_size=3,strides=2,padding='same',activation='relu'),
                      
                         Conv2D(64,kernel_size=3,activation='relu'),
                      
                         GlobalAveragePooling2D(),
                      
                         Dense(embedding_dim) 

])

print(embedding.summary())

### Creating Siamese Network Architecture With Triplets Of Anchor, Positive, and Negative Image Patches as Input

In [ ]:
from tensorflow.keras.applications import resnet

from tensorflow.keras import applications

from tensorflow.keras import layers,losses,optimizers,metrics,Model

import tensorflow as tf

from tensorflow.keras.utils import plot_model

patch_size=32

anchor_input=layers.Input(name='anchor',shape=(patch_size,patch_size)+(3,))

positive_input=layers.Input(name='positive',shape=(patch_size,patch_size)+(3,))

negative_input=layers.Input(name='negative',shape=(patch_size,patch_size)+(3,))

embedding_anchor=embedding(anchor_input)
embedding_positive=embedding(positive_input)
embedding_negative=embedding(negative_input)

#output=layers.concatenate([embedding_anchor,embedding_positive,embedding_negative])
output=layers.concatenate([embedding_positive,embedding_negative])

#model=Model([anchor_input,positive_input,negative_input],output)
model=Model([positive_input,negative_input],output)

#plot_model(model,to_file='D:/Nafiseh/flood_proposal/siamese_model_contrastive_loss_intensity_coh.png',show_shapes=True,show_layer_names=True)

# Function Creating Triplets Of Anchor, Positive, and Negative Patches

In [ ]:
import random

from keras.preprocessing.image import ImageDataGenerator


def create_batch_data(train_resampled,label,batch_size):
    
        
    def data_augmentation(data,image_count=1):

        augmented_images=[]
        
        patch_size=32

        datagen=ImageDataGenerator(horizontal_flip=True)

        it=datagen.flow(np.expand_dims(data,axis=0),batch_size=1)

        for i in range(image_count):
    
            augmented_images.append(np.squeeze(it.next(),axis=0))
        
        return(np.stack(augmented_images)) 
    
    anchors,positives,negatives=np.zeros((batch_size,patch_size,patch_size,3)),np.zeros((batch_size,patch_size,patch_size,3)),np.zeros((batch_size,patch_size,patch_size,3))

    positive_idx=np.squeeze(np.where(np.array(label)!=1)) #background 

        
    negative_idx=np.squeeze(np.where(np.array(label)==1))
    
    for i in range(batch_size):
        
        
        index=positive_idx[random.randint(0,len(positive_idx)-1)]

        positive=train_resampled[index]  
        
        anchor=data_augmentation(positive)
        
        label_=label[index]
        
        negative=train_resampled[negative_idx[random.randint(0,len(negative_idx)-1)]]
        
        anchors[i],positives[i],negatives[i]=anchor,positive,negative 
    
    return [anchors,positives,negatives]


# Triplet Loss

In [ ]:
import tensorflow as tf

def Triplet_Loss(margin, embedding_dim):
    
    def get_loss(output_true,output_pred):
        
        anchor_output=output_pred[:,:embedding_dim]
        
        positive_output=output_pred[:,embedding_dim:2*embedding_dim]
        
        negative_output=output_pred[:,2*embedding_dim:]
        
        dp=tf.reduce_sum(tf.square(anchor_output-positive_output),axis=1)
        
        dn=tf.reduce_sum(tf.square(anchor_output-negative_output),axis=1)
        
        return tf.maximum(dp-dn+margin,0)
        
    return get_loss    

# Contrastive Loss

In [ ]:
def Contrastive_Loss(margin,embedding_dim,tav): #tav:maximum acceptable distance between similar samples
    
    def get_loss(output_true,output_pred):
        
        input1=output_pred[:,:embedding_dim]
        
        input2=output_pred[:,embedding_dim:2*embedding_dim]
        
        d=tf.reduce_sum(tf.square(input1-input2))
        
        return(output_true*tf.maximum(margin-d,0)+(1-output_true)*tf.maximum(d-tav,0))  
    
    return get_loss

# Weighted Double Margin Contrastive Loss (WDMCL)

In [ ]:
def Weighted_Double_Margin_Contrastive_Loss(embedding_dim,w1,w2 ,m1=0.9,m2=0.45):
    
    def get_loss(output_true,output_pred):

        
        input1=output_pred[:,:embedding_dim]
    
        input2=output_pred[:,embedding_dim:2*embedding_dim]
        
        d=tf.reduce_sum(tf.square(input1-input2))
        
        return(0.5*((w1*(1-output_true)*(tf.maximum(d-m1,0))**2)+(w2*output_true*(tf.maximum(m2-d,0))**2)))
    
    return(get_loss)       

In [ ]:
def Data_Generator(train_resampled,label,batch_size,embedding_dim,mode):
    
    while True:
    
        x=create_batch_data(train_resampled,label,batch_size)
        
        if mode=='triplet':
        
            y=np.zeros((batch_size,3*embedding_dim))
            
        else:
            
            x=x[1:]
            
            y=np.zeros((batch_size,2*embedding_dim))
        
        yield x,y

# Apply PCA transfrom on training data

In [ ]:
from sklearn.decomposition import PCA


def apply_pca_transform(data,n_components=3,patch_size=32):

    def make_2d_feature_map_1d(data):

        data_ravel=np.zeros((data.shape[0],data.shape[1]*data.shape[2],data.shape[3]))

        for i in range(data.shape[0]):

            for j in range(data.shape[3]):

                 data_ravel[i,:,j]=np.ravel(data[i,:,:,j])

        return(data_ravel)      


    data_1d=make_2d_feature_map_1d(data)

    data_transformed=[]
    data_transformed_2d=[]

    pca=PCA(n_components=3)

    for k in range(data_1d.shape[0]):

        data_transformed.append(pca.fit_transform(data_1d[k,:,:]))

        data_transformed_2d.append(np.reshape(data_transformed[k],(patch_size,patch_size,n_components)))
      
    return(np.array(data_transformed_2d))

train_resampled__=apply_pca_transform(train_resampled_)
val_resampled__=apply_pca_transform(val_resampled)

# Fitting The Siamese Model and Setting Training Parameters

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam, SGD


batch_size,embedding_dim, epochs=50, 256, 200

lr_schedule=ExponentialDecay(0.0001,decay_rate=0.8,decay_steps=10000)

#model.compile(loss=Triplet_Loss(margin=0.9, embedding_dim=embedding_dim),optimizer=Adam(learning_rate=lr_schedule))

        
w1=1/(len(train_label[train_label==0])) # weight for unchanged pixels
        
w2=1/(len(train_label[train_label==1])) # weight for changed pixels


#Weighted_Double_Margin_Contrastive_Loss(embedding_dim,w1,w2,m1=0.9,m2=0.45)
model.compile(loss=Contrastive_Loss(embedding_dim=embedding_dim,margin=0.01,tav=0.01),optimizer=Adam(learning_rate=lr_schedule))

es=EarlyStopping(monitor='val_loss',patience=30,restore_best_weights=True)

mch=ModelCheckpoint(filepath='D:/Nafiseh/flood_proposal/leverkhuzen_model.h5',save_best_only=True)

val_data=create_batch_data(val_resampled__,val_label,val_resampled__.shape[0])

val_data=val_data[1:] #omitting the anchor images


history=model.fit(Data_Generator(train_resampled__,train_label,batch_size,embedding_dim,'contrastive'),steps_per_epoch=int(train_resampled__.shape[0]/batch_size),epochs=epochs,callbacks=[es,mch],validation_data=(val_data,val_label))

In [ ]:
import numpy as np

files=np.load('D:/Nafiseh/flood_proposal/leverkhuzen_saved_arrays.npz')

train_resampled__,train_label,val_resampled__,val_data,val_label,embedding_dim=files['arr_0'],files['arr_1'],files['arr_2'],files['arr_3'],files['arr_4'],files['arr_5']

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])

plt.plot(history.history['val_loss'])

plt.xlabel('Epoch')

plt.ylabel('Tripplet Loss')

plt.legend(['Train','Validation'])

plt.show()

In [ ]:
model.save('D:/Nafiseh/flood_proposal/siamese_intensity_coh_leverkhuzen.h5')

In [ ]:
from tensorflow.keras.models import load_model

#model=load_model('D:/Nafiseh/flood_proposal/siamese_intensity_coh_partial_TR_contrastive_loss_downsampling.h5',compile=False)
model=load_model('D:/Nafiseh/flood_proposal/siamese_intensity_coh_leverkhuzen.h5',compile=False)

In [ ]:
import pickle

with open('D:/Nafiseh/flood_proposal/training history_resnet50_intensity_coh_leverkhuzen.pckl', 'wb') as hist:
    
    pickle.dump(history.history,hist)

In [ ]:
import pickle

f=open('D:/Nafiseh/flood_proposal/training history_resnet50_intensity_coh_partial_TR_weighted_contrastive_loss.pckl','rb')

history=pickle.load(f)

f.close()

# Prediction

In [ ]:
from sklearn.utils import shuffle

from tensorflow.keras.layers import LeakyReLU

from tensorflow.keras import layers,Model,regularizers

embedding_dim=256

patch_size=32

input_layer=layers.Input(shape=(embedding_dim,))

flatten1=layers.Flatten()(input_layer)

dense1=layers.Dense(64)(flatten1)

LR1=LeakyReLU(alpha=0.1)(dense1)

dense2=layers.Dense(64)(LR1)

LR2=LeakyReLU(alpha=0.1)(dense2)

drop1=layers.Dropout(0.1)(LR2)

#dense3=layers.Dense(128,kernel_regularizer=regularizers.l1(0.0001))(drop1)

#LR3=LeakyReLU(alpha=0.1)(dense3)

#drop2=layers.Dropout(0.5)(LR3)

#dense4=layers.Dense(64,kernel_regularizer=regularizers.l1(0.0001))(drop2)

#LR4=LeakyReLU(alpha=0.1)(dense4)

dense5=layers.Dense(32)(drop1)

LR5=LeakyReLU(alpha=0.1)(dense5)

prediction_layer=layers.Dense(2,activation='softmax')(LR5)

model1=Model(input_layer,prediction_layer,name='prediction_model')

train_data=create_batch_data(train_resampled__,train_label,train_resampled__.shape[0])

train_data=train_data[1:] #omitting the anchor images

##train_label_pos,train_label_neg=train_label[np.where(train_label==0)[0]],train_label[np.where(train_label==1)[0]]

train_prediction=model.predict(train_data)

val_prediction=model.predict(val_data)

# Selecting A Subset Of Train And Validation Data (Memory Issue)

In [ ]:
def subset(data,label):

    random.seed(1)

    pos_idx,neg_idx=np.where(np.array(label)==1)[0],np.where(np.array(label)==0)[0]

    pos_idx_subset,neg_idx_subset=random.sample(list(pos_idx),500),random.sample(list(neg_idx),500)

    test_idx_subset=pos_idx_subset+neg_idx_subset

    data_s=[data[i,:] for i in test_idx_subset]

    label_s=[label[j] for j in test_idx_subset]
    
    return(np.stack(data_s),np.stack(label_s))

#train_prediction_s,train_label_s=subset(train_prediction,train_label)

#val_prediction_s,val_label_s=subset(val_prediction,val_label)

# Senario 1: Difference Feature Vectors As Trainin Data to the Network

In [ ]:
train_diff_pos=train_prediction[:,:embedding_dim]-train_prediction[:,embedding_dim:2*embedding_dim]

train_diff_neg=train_prediction[:,:embedding_dim]-train_prediction[:,2*embedding_dim:]

train_diff=np.concatenate((train_diff_pos,train_diff_neg),axis=0)

train_diff_label=np.concatenate((np.zeros((len(train_diff_pos),)),np.ones((len(train_diff_neg),))),axis=0)

val_diff_pos=val_prediction[:,:embedding_dim]-val_prediction[:,embedding_dim:2*embedding_dim]

val_diff_neg=val_prediction[:,:embedding_dim]-val_prediction[:,2*embedding_dim:]

val_diff=np.concatenate((val_diff_pos,val_diff_neg),axis=0)

val_diff_label=np.concatenate((np.zeros((len(val_diff_pos),)),np.ones((len(val_diff_neg),))),axis=0)

# Senario 2: The Original Feature Vectors As Training Data to the Network

In [ ]:
anchor_f=train_prediction[:,:embedding_dim]
pos_f=train_prediction[:,embedding_dim:2*embedding_dim]
neg_f=train_prediction[:,2*embedding_dim:]
train_pos,train_neg=np.concatenate((anchor_f,pos_f),axis=0),neg_f
train_fv=np.concatenate((train_pos,train_neg),axis=0)

pos_num=len(anchor_f)+len(pos_f)
neg_num=len(neg_f)
train_fv_label=np.concatenate((np.zeros((pos_num,)),np.ones((neg_num,))),axis=0)

anchor_f_val,pos_f_val=val_prediction[:,:embedding_dim],val_prediction[:,embedding_dim:2*embedding_dim]
neg_f_val=val_prediction[:,2*embedding_dim:]
val_pos,val_neg=np.concatenate((anchor_f_val,pos_f_val),axis=0),neg_f_val
val_fv=np.concatenate((val_pos,val_neg),axis=0)
pos_num_val=len(anchor_f_val)+len(pos_f_val)
neg_num_val=len(neg_f_val)
val_fv_label=np.concatenate((np.zeros((pos_num_val,)),np.ones((neg_num_val,))),axis=0)

In [ ]:
pos_f=train_prediction[:,:embedding_dim]
neg_f=train_prediction[:,embedding_dim:]
train_fv=np.concatenate((pos_f,neg_f),axis=0)

pos_num=len(pos_f)
neg_num=len(neg_f)
train_fv_label=np.concatenate((np.zeros((pos_num,)),np.ones((neg_num,))),axis=0)

pos_f_val,neg_f_val=val_prediction[:,:embedding_dim],val_prediction[:,embedding_dim:]
val_fv=np.concatenate((pos_f_val,neg_f_val),axis=0)
pos_num_val=len(pos_f_val)
neg_num_val=len(neg_f_val)
val_fv_label=np.concatenate((np.zeros((pos_num_val,)),np.ones((neg_num_val,))),axis=0)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.optimizers import Adam,SGD,Adadelta

from tensorflow.keras.losses import CategoricalCrossentropy

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.optimizers.schedules import ExponentialDecay

#anchor_embedding_train,positive_embedding_train,negative_embedding_train=train_prediction[:,:256],train_prediction[:,256:2*256],train_prediction[:,2*256:3*256]

#embedding_t,label_t=shuffle(np.concatenate((positive_embedding_train,negative_embedding_train),axis=0),np.concatenate((train_label_pos,train_label_neg),axis=0),random_state=2)

lr_Schedule1=ExponentialDecay(0.0001,decay_rate=0.8,decay_steps=1000)

es1=EarlyStopping(monitor='val_loss',min_delta=1e-4,patience=100,restore_best_weights=True)

model1.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate=lr_Schedule1),metrics=['accuracy'])

history1=model1.fit(train_fv,to_categorical(train_fv_label,num_classes=2),batch_size=80, epochs=300,callbacks=[es1],validation_data=(val_fv,to_categorical(val_fv_label,num_classes=2)))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))

plt.plot(history1.history['loss'])

plt.plot(history1.history['val_loss'])

plt.xlabel('Epoch')

plt.ylabel('Binary Cross Entropy')

plt.legend(['Train','Validation'])


plt.figure(figsize=(10,5))

plt.plot(history1.history['accuracy'])

plt.plot(history1.history['val_accuracy'])

plt.xlabel('Epoch')

plt.ylabel('Accuracy')

plt.legend(['Train','Validation'])

plt.show()

In [ ]:
model1.save('D:/Nafiseh/flood_proposal/prediction_layer_model_intensity_coh_leverkhuzen.h5')

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LeakyReLU

model1=load_model('D:/Nafiseh/flood_proposal/prediction_layer_model_intensity_coh_leverkhuzen.h5',compile=False)

In [ ]:
import pickle

with open('D:/Nafiseh/flood_proposal/prediction_layer_model_intensity_coh_leverkhuzen.pckl','wb') as hist:
    
    pickle.dump(history1.history,hist)

In [ ]:
import pickle

f=open('D:/Nafiseh/flood_proposal/prediction_layer_model_intensity_coh_partial_TR_weighted_contrastive_loss.pckl','rb')

history1=pickle.load(f)

f.close()

# Training Resnet50 From Scratch

In [ ]:
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam, SGD


batch_size,embedding_dim, epochs=50, 256, 200
es=EarlyStopping(monitor='val_loss',patience=50,restore_best_weights=True)

lr_schedule=ExponentialDecay(0.0001,decay_rate=0.8,decay_steps=10000)

model.compile(loss=Triplet_Loss(margin=0.9, embedding_dim=embedding_dim),optimizer=Adam(learning_rate=lr_schedule))

es1=EarlyStopping(monitor='val_loss',min_delta=1e-4,patience=100,restore_best_weights=True)

embedding.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate=lr_Schedule),metrics=['accuracy'])

history1=embedding.fit(train_resampled_,to_categorical(train_label,num_classes=2),batch_size=80, epochs=300,callbacks=[es1],validation_data=(val_resampled,to_categorical(val_label,num_classes=2)))

In [ ]:
def create_change_map(model,model1,train_data,preflood_resampled,test_resampled,label_test_new,embedding_dim):

    test_label_p=[]

    for i in range(len(test_resampled)):

        #preflood_resampled_expanded=np.expand_dims(preflood_resampled[i],axis=2)
        preflood_resampled_expanded=preflood_resampled
        
        embedding_input=np.expand_dims(np.concatenate((preflood_resampled_expanded[i],test_resampled[i]),axis=2),axis=0)
        
        if i>=len(train_data[0]): #end of the train data
            
            if i%len(train_data[0])==0:
                
               k=0 

            train_anchor=np.expand_dims(train_data[0][k,:,:,:],axis=0)
            
            train_pos=np.expand_dims(train_data[1][k,:,:,:],axis=0)
            
            train_neg=np.expand_dims(train_data[2][k,:,:,:],axis=0)
            
            k+=1
        
        else:
            

            train_anchor=np.expand_dims(train_data[0][i,:,:,:],axis=0)

            train_pos=np.expand_dims(train_data[1][i,:,:,:],axis=0)

            train_neg=np.expand_dims(train_data[2][i,:,:,:],axis=0)

        if label_test_new[i]==1: #change

            embedding=model.predict([train_anchor,train_pos,embedding_input])
            
        elif label_test_new[i]==0: #no change

            embedding=model.predict([train_anchor,embedding_input,train_neg])
        
        #prediction=np.expand_dims(prediction,axis=0)
        
        if label_test_new[i]==1: #change

            test_label_p.append(np.argmax(model1.predict(embedding[:,2*embedding_dim:])))
        
        elif label_test_new[i]==0: #no change
            
            test_label_p.append(np.argmax(model1.predict(embedding[:,embedding_dim:2*embedding_dim])))
            
                            
    return(test_label_p) 

intensity_array_vv_expanded=np.expand_dims(intensity_array_vv[:,:,0],axis=2)

preflood_label=np.zeros((1500,1500))
                            
preflood_array_sample,preflood_label_pad=prepare_test_data(intensity_array_vv_expanded,preflood_label)

preflood_resampled,_=resample_data(preflood_array_sample,preflood_label_pad,'preflood') 

test_label_p=create_change_map(model,model1,train_data,preflood_resampled,test_resampled,label_test_new,embedding_dim)                            

In [ ]:
def create_change_map(model,model1,train_data,test_resampled,label_test_new,embedding_dim):

    test_label_p=[]
    
    #preflood_resampled_expanded=preflood_resampled

    for i in range(len(test_resampled)):
        
        embedding_input=np.expand_dims(test_resampled[i],axis=0)
        
        if i>=len(train_data[0]): #end of the train data
            
            if i%len(train_data[0])==0:
                
               k=0 
            
            train_pos=np.expand_dims(train_data[0][k,:,:,:],axis=0)
            
            train_neg=np.expand_dims(train_data[1][k,:,:,:],axis=0)
            
            k+=1
        
        else:

            train_pos=np.expand_dims(train_data[0][i,:,:,:],axis=0)

            train_neg=np.expand_dims(train_data[1][i,:,:,:],axis=0)

        if label_test_new[i]==1: #change

            embedding=model.predict([train_pos,embedding_input])
            
        elif label_test_new[i]==0: #no change

            embedding=model.predict([embedding_input,train_neg])
        
        #prediction=np.expand_dims(prediction,axis=0)
        
        if label_test_new[i]==1: #change

            test_label_p.append(np.argmax(model1.predict(embedding[:,embedding_dim:])))
        
        elif label_test_new[i]==0: #no change
            
            test_label_p.append(np.argmax(model1.predict(embedding[:,:embedding_dim])))
            
                            
    return(test_label_p) 

#index=[]

#for i in range(0,int((len(test_sample)*2)/(col1+2))-1,2):
    
    #index.append(np.arange(i*(int((col1+2)/2)),(i+1)*(int((col1+2)/2))))
    

#f_index=np.ravel(index)  

#f_index=np.delete(f_index,-1)

#row1=stack_test_int_coh.shape[0]
#col1=stack_test_int_coh.shape[1]

#ts_array,lt_array=np.array(test_sample),np.array(label_test)

#test_resampled,label_test_new=resample_data(test_sample[s],lt_array[s],'Test')

chunks=1000

test_label_pp=[]

for s in range(0,len(test_sample),chunks):
    
    test_resampled,label_test_new=resample_data(test_sample[s:s+chunks],label_test[s:s+chunks],'Test')

    test_resampled__=apply_pca_transform(test_resampled)

#test_label_p=create_change_map(embedding,model1,train_data,test_resampled__,label_test_new,embedding_dim) #embedding do the same job as model

    test_label_pp.append(create_change_map(model,model1,train_data,test_resampled__,label_test_new,embedding_dim)) 

In [ ]:
import numpy as np

np.save('D:/Nafiseh/flood_proposal/leverkhuzen_change_labels_intensity_coh.npy',test_label_p)

#test_label_p=np.load('D:/Nafiseh/flood_proposal/test_label_p_leverkhuzen_2.npy')

In [ ]:
nrows,ncols=992,2038

change_map=np.reshape(np.concatenate((np.ravel(np.stack(test_label_pp[:-1])),test_label_pp[-1])),(nrows,ncols))

plt.imshow(change_map,cmap=plt.cm.get_cmap('binary').reversed())

plt.axis('off')

plt.show()

In [ ]:
import numpy as np
np.save('D:/Nafiseh/flood_proposal/leverkhuzen_change_labels_resnet_50_intensity_coh.npy', test_label_pp)
#test_label_p=np.load('D:/Nafiseh/flood_proposal/gatineau_change_labels.npy')
#test_label_p=np.load('D:/Nafiseh/flood_proposal/gatineau_change_labels_intensity_coh_partial_TR.npy')

In [ ]:
def normalize_rgb(image):
    
    for i in range(image.shape[2]):

        image[:,:,i]=np.multiply((image[:,:,i]-np.min(image[:,:,i]))/(np.max(image[:,:,i])-np.min(image[:,:,i])),255)
  
    return(image)    

# Visualizing The Result

In [ ]:
from osgeo import gdal,osr

import matplotlib.pyplot as plt

import utm

import numpy as np

path='D:/Nafiseh/flood_proposal'

#stack_image_name='subset_0_of_S1A_IW_GRDH_1SDV_Cal_Spk_TC_Stack.tif'

#stack_image_name='subset_1_of_S1A_IW_GRDH_1SDV_20190408T225218_20190507T23009_Cal_Spk_TC_Stack_2_subset.tif'

stack_image_name='collocate_S1B_int_coh.tif'

#Rapid_Eye_Image_name='0428_clip_3.tif'

gt_name='hydrography_area_mask_leverkhuzen_clip.tif'

gt=read_as_array(path,gt_name)

#image_array=read_as_array(path,Rapid_Eye_Image_name)

ds=gdal.Open(path+'/'+stack_image_name)

#rapid_eye_ds=gdal.Open(path+'/'+Rapid_Eye_Image_name)

ulx,xres,xskew,uly,yskew,yres=ds.GetGeoTransform()

#uly_eye,yres_eye,yskew_eye,ulx_eye,xskew_eye,xres_eye=rapid_eye_ds.GetGeoTransform()

#row,col=1501,1501

#row,col=1379-1011+1,1055-505+1

#row,col=301,301

row,col=992, 2038

#row_ul_new,col_ul_new=780,0
row_ul_new,col_ul_new=0, 0

#row_lr_new,col_lr_new=1080,300
row_lr_new,col_lr_new=992, 2038


gt=read_as_array(path,gt_name)


def find_new_pixel_cor(row_new,col_new,ulx,uly,ulx_eye,uly_eye,xres,yres,xres_eye,yres_eye):

    x_new=col_new*xres+ulx

    y_new=row_new*yres+uly

    utm_cor=utm.from_latlon(y_new,x_new)

    y_new_utm,x_new_utm=utm_cor[0],utm_cor[1]

    row_eye=np.int(np.floor((y_new_utm-uly_eye)/yres_eye))

    col_eye=np.int(np.floor((x_new_utm-ulx_eye)/xres_eye))
    
    return(x_new,y_new,row_eye,col_eye)

#x_new,y_new,row_ul_eye,col_ul_eye=find_new_pixel_cor(row_ul_new,col_ul_new,ulx,uly,ulx_eye,uly_eye,xres,yres,xres_eye,yres_eye)

#_,_,row_lr_eye,col_lr_eye=find_new_pixel_cor(row_lr_new,col_lr_new,ulx,uly,ulx_eye,uly_eye,xres,yres,xres_eye,yres_eye)

#RGB_image_array=image_array[:row_lr_eye+1,col_ul_eye:col_lr_eye+1,[2,1,0]]

#ul_x_new,ul_y_new=x_new,y_new

#BBOX=[ul_x_new, ul_x_new+xres*300, ul_y_new+yres*300, ul_y_new]
BBOX=[6.9, 6.9+xres*2038, 50.983+yres*992, 51.072]

#change_map=np.reshape(test_label_p,(row,col))

orig_map=plt.cm.get_cmap('binary')

reversed_map=orig_map.reversed()

fig,ax=plt.subplots(1,2,figsize=(10,10))

plt.setp(ax, xticks=np.round(np.arange(BBOX[0],BBOX[1],0.05),2), yticks=np.round(np.arange(BBOX[2],BBOX[3],0.06),3))

ax[0].imshow(change_map, extent=BBOX, aspect='equal',cmap=reversed_map)

#ax[0].set_title('Change Map \n 07/05/2019',fontname='Times New Roman', fontweight='bold', fontsize=14)
#ax[0].set_title('Change Map \n 18/07/2019',fontname='Times New Roman', fontweight='bold', fontsize=14)
ax[0].set_title('Flood Map',fontname='Times New Roman', fontweight='bold', fontsize=14)

ax[1].imshow(gt,extent=BBOX, aspect='equal', cmap=reversed_map)

ax[1].set_title('Ground Truth Mask',fontname='Times New Roman', fontweight='bold',fontsize=14)

#ax[2].imshow(np.divide(RGB_image_array,(8,8,8)).astype('uint8'), extent=BBOX)

#ax[2].set_title('Rapid Eye RGB image \n 28/04/2019', fontname='Times New Roman', fontweight='bold',fontsize=14)

fig.tight_layout(pad=3)

#plt.savefig('D:/Nafiseh/flood_proposal/intensity_coh_contrastive_loss_resnet50_leverkhuzen.tif')

plt.show()

# Save Change Map and Ground Truth Map as Geo Tiff

In [ ]:
from osgeo import gdal, osr

def Write_array_as_geotiff(array,ul_x_new,ul_y_new,raster_name):

    ds=gdal.Open('D:/Nafiseh/flood_proposal/collocate_S1B_int_coh.tif')

    ulx,xres,xskew,uly,yskew,yres=ds.GetGeoTransform()

    geotransform=(ul_x_new, xres, xskew, ul_y_new, yskew, yres)

    raster=gdal.GetDriverByName('GTiff').Create(raster_name,992,2038,1,gdal.GDT_Float32)

    raster.SetGeoTransform(geotransform)

    srs=osr.SpatialReference()

    srs.ImportFromEPSG(4326)
    
    raster.GetRasterBand(1).WriteArray(array)
    
    raster.GetRasterBand(1).SetNoDataValue(0)
    
    raster.FlushCache() ##saves to disk
    
    raster=None
    
    ds=None


Write_array_as_geotiff(change_map,0,0,'D:/Nafiseh/flood_proposal/CHM_contrastive_Loss_resnet50_intensity_coh.tif')

#Write_array_as_geotiff(label_test_new_2D,ul_x_new,ul_y_new,'D:/Nafiseh/flood_proposal/Change_mask_07052019.tif')

In [ ]:
test_resampled_s,label_test_s=subset(test_resampled,label_test_new)

test_data=create_batch_data(test_resampled_s,label_test_s,test_resampled_s.shape[0])

prediction=model.predict(test_data)

diff_pr_pos=prediction[:,:embedding_dim]-prediction[:,embedding_dim:2*embedding_dim]

diff_pr_neg=prediction[:,:embedding_dim]-prediction[:,2*embedding_dim:]

diff_pr=np.concatenate((diff_pr_pos,diff_pr_neg),axis=0)

#anchor_embedding_test,positive_embedding_test,negative_embedding_test=prediction[:,:256],prediction[:,256:2*256],prediction[:,2*256:3*256]

test_label_p=np.argmax(model1.predict(diff_pr),axis=1)

#dist_pos=np.square(np.expand_dims(np.sum(np.power(diff_pr_pos,2),axis=1),axis=1))
#dist_neg=np.square(np.expand_dims(np.sum(np.power(diff_pr_neg,2),axis=1),axis=1))

#diff_power=np.concatenate((dist_pos,dist_neg),axis=1)

#test_label_p2=np.argmin(diff_power,axis=1)

In [ ]:
test_resampled_s,label_test_s=subset(test_resampled,label_test_new)

test_data=create_batch_data(test_resampled_s,label_test_s,test_resampled_s.shape[0])

prediction=model.predict(test_data)

pr_anchor,pr_pos,pr_neg=prediction[:,:embedding_dim],prediction[:,embedding_dim:2*embedding_dim],prediction[:,2*embedding_dim:]

pr=np.concatenate((pr_anchor,pr_pos,pr_neg),axis=0)

test_label_p=np.argmax(model1.predict(pr),axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix

from sklearn.metrics import classification_report

import seaborn as sns

plt.figure(figsize=(5,5))

change_map_r=zoom(change_map,[gt.shape[0]/change_map.shape[0],gt.shape[1]/change_map.shape[1]],order=1)

sns.heatmap(confusion_matrix(np.ravel(gt),np.ravel(change_map_r)),annot=True,fmt='d')

#plt.savefig('D:/Nafiseh/flood_proposal/confusion_matrix_contrastive_loss_resnet50_leverkhuzen.tif')

text_file=open('D:/Nafiseh/flood_proposal/classification_report_contrastive_loss_resnet50_leverkhuzen.txt','a')

print(classification_report(np.ravel(change_map_r),np.ravel(gt)))

text_file.close()